## Load necessary modules

In [ ]:
import os
os.chdir('/home/jasper/git/CEIG/keras-retinanet')

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

## Load RetinaNet model

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('EarlyFusionDay', 'resnet101_pascal_53.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet101')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'plum'}

In [ ]:
# load images
print("Running tests")
print("If running slow, did CUDA get loaded?")

datasetPath = '/home/jasper/Datasets/Day/'
testPath = datasetPath + 'ImageSets/Main/test.txt'
imgsPath = datasetPath + 'JPEGImages/'
depthsPath = datasetPath + 'DepthImages/'

visualise = False

testFile = open(testPath, 'r')

resultPath = os.path.split(model_path)[0]
resultPath = os.path.join(resultPath, 'detections.txt')
resultFile = open(resultPath,'w')

firstStart = time.time()
cont = testFile.readlines()
for i in range(0, len(cont)):
    print("detecting image: {}".format(cont[i].rstrip()))
    
    imgPath = imgsPath + cont[i].rstrip() + '.jpg'
    image = read_image_bgr(imgPath)
    depth = depthsPath + cont[i].rstrip() + '.npy'
    depth = np.load(depth)
    depth = np.expand_dims(depth, axis=2)
    rgbd = np.concatenate((image, depth), axis=2)
    
    print(rgbd.shape)

    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    rgbd = preprocess_image(rgbd)
    rgbd, scale = resize_image(rgbd)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(rgbd, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.01:
            break
                
        bb = np.array(box).astype(int)
        dataline = cont[i].rstrip() + ' ' + str(score) + ' ' + str(bb[0]) + ' ' + str(bb[1]) + ' ' + str(bb[2]) + ' ' + str(bb[3]) + '\n' 
        resultFile.write(dataline)
        #print(dataline)
        
        
        
    if(visualise):
        # visualize detections
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            # scores are sorted so we can break
            if score < 0.5:
                break

            color = label_color(label)

            b = box.astype(int)
            draw_box(draw, b, color=color)

            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

        plt.figure(figsize=(15, 15))
        plt.axis('off')
        plt.imshow(draw)
        plt.show()
    
print("DONE~!!")
print("Detected on {} images in {} sec".format(len(cont), time.time() - firstStart))